In [21]:
import pandas as pd
import numpy as np
import string
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import recall_score, precision_recall_curve, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')

from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vlads\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
# Загрузка данных
data = pd.read_csv(r"C:\__test\py_test\data\482reviews.csv")

In [23]:
X_train, X_test, y_train, y_test = train_test_split(data['review'], data['assessment'], test_size = 0.25, random_state = 1)

In [24]:
# Предобработка текста

snowball = SnowballStemmer(language = "russian")
russian_stop_words = stopwords.words("russian")

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language = "russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [25]:
vectorizer = TfidfVectorizer(tokenizer = lambda x: tokenize_sentence(x, remove_stop_words=True), token_pattern=None)

In [26]:
features = vectorizer.fit_transform(X_train)

In [27]:
model = LogisticRegression(random_state = 0)
model.fit(features, y_train)

LogisticRegression(random_state=0)

In [28]:
X = vectorizer.fit_transform(X_train)
y_pred = model.predict(X)

In [29]:
model_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer()),
    ("model", LogisticRegression(random_state=0))])

In [30]:
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('model', LogisticRegression(random_state=0))])

In [31]:
# Здесь у меня ошибка так как не знаю откуда берется model_B поэтому я просто закомментировал
# Байес
# model_B.fit(features, y_train)
model_pipeline_B = Pipeline([
    ("vectorizer", TfidfVectorizer()),
    ("model", MultinomialNB())])

In [32]:
model_pipeline_B.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()), ('model', MultinomialNB())])

In [33]:
model_pipeline

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('model', LogisticRegression(random_state=0))])

In [34]:
print(model_pipeline.predict(["Очень хорошая игра, с удовольствием в неё поиграл"]))
print(model_pipeline.predict(["плохая игра, не рекомендуется, зря потратил деньги"]))

[1]
[0]


In [36]:
import dill

# # Сохраняем модели

# without pipeline
with open(r"C:\__project\evaluation_of_reviews\src\models\logistic_regression_classifier.pickle", 'wb') as file:
    dill.dump({'model': model_pipeline}, file)
# with pipeline
with open(r"C:\__project\evaluation_of_reviews\src\models\naive_bayes_classifier.pickle", 'wb') as file:
    dill.dump({'model': model_pipeline_B}, file)